<a href="https://colab.research.google.com/github/VictorUceda/TrackNet-Padel/blob/main/TrackNet_Padel_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Thu Jun  9 15:28:12 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Inicializacion

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
import os 
os.chdir('/content/')

!git clone https://github.com/VictorUceda/TrackNet-Padel.git
#!git clone https://github.com/Chang-Chia-Chi/TrackNet-Badminton-Tracking-tensorflow2
os.chdir('/content/TrackNet-Padel/')
!git pull

Cloning into 'TrackNet-Padel'...
remote: Enumerating objects: 1076, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 1076 (delta 0), reused 0 (delta 0), pack-reused 1073
Receiving objects: 100% (1076/1076), 1.04 GiB | 31.52 MiB/s, done.
Resolving deltas: 100% (467/467), done.
Checking out files: 100% (314/314), done.
Already up to date.


In [ ]:
!pip3 install filterpy sktime piexif imutils Pillow focal_loss

### Predict

In [ ]:
weights_path = '/content/gdrive/MyDrive/PadelModels/TrackNetWeights/TrackNet-PadelClean4'
for i in range(7):
  video = 'raw_data/points/cleanVal/PremierRoma_Semis_LGBC_3_42_'+str(i)+'.mp4'
  !python predict.py --video_path={video} --load_weights={weights_path}

2022-06-02 13:55:03.763298: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Load weights successfully
Number of frames in video and dictionary are not the same!
Fail to load, predict only.
Beginning predicting......
Done......


### Test accuracy

In [28]:
file_name = 'Final_PremierQatar_Tie'
file_name = 'cleanVal/PremierRoma_Semis_LGBC_3_42'
weights_path = '/content/gdrive/MyDrive/PadelModels/TrackNetWeights2/TrackNet-SoloPadelCleanAll-480p'
point_n = 0
for point_n in range(7):
  video = 'raw_data/points/'+file_name+"_"+str(point_n)+".mp4"
  print(video)
  csv = 'raw_data/points/'+file_name+"_"+str(point_n)+".csv"
  res = 'raw_data/points/'+file_name+"_"+str(point_n)+".resPadel.txt"
  !python ../TrackNet-Padel/predict.py --video_path={video} --label_path={csv} --HEIGHT=480 --WIDTH=720 --load_weights={weights_path} # 2>&1 | tee {res}

raw_data/points/cleanVal/PremierRoma_Semis_LGBC_3_42_0.mp4
2022-06-09 12:10:17.573969: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Load weights successfully
Load csv file successfully
Beginning predicting......
Number of true positive: 196
Number of true negative: 20
Number of false positive FP1: 15
Number of false positive FP2: 1
Number of false negative: 16
Accuracy: 0.8709677419354839
Precision: 0.9245283018867925
Recall: 0.9245283018867925
Total Time: 32.80319046974182
(ACC + Pre + Rec)/3: 0.906674781903023
Done......
raw_data/points/cleanVal/PremierRoma_Semis_LGBC_3_42_1.mp4
2022-06-09 12:11:00.879322: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Load weights successfully
Load csv file successfu

### Train Padel

In [ ]:
!mv raw_data/*.* raw_data/points/
for point_n in range(9):
  files = 'raw_data/points/clean/*_'+str(point_n)+'\.*'
  print(files)
  !mv {files} raw_data/

In [ ]:
!rm -rf train_data/*
!python video2img.py --HEIGHT=480 --WIDTH=720 --sigma=3 

#### Train sequencial (error en el orden de las imagenes)
*   TrackNet-PadelClean2: Entrenado solo con padel, pocos frames (5000) limpios (solo fragmentos con la bola en juego). --lr=10
*   TrackNet-PadelClean3: Continuación, 50 epochs --batch_size=5, con 7k frames (alcanza 0.956 en la validación. - loss:1.20e-4)
*   TrackNet-PadelClean4: Continuación, +50epochs --bacth_size=5, con 12k frames, --lr=1 (alcanza 0.94 en la validación. - loss:1.1*   TrackNet-PadelClean5: Continuación, +50epochs (800 steps), con 20k frames, --lr=1 (alcanza 0.93 en la validación. - loss:1.09e-4)8e-4)
-----
*   TrackNet-PadelCleanAll: Solo padel CleanDS, 25epochs (1k steps), con 20k frames, --lr=10 (alcanza 0.82 en la validación. - loss:1.80e-4)
*   TrackNet-PadelCleanAll1: Continuación, 50epochs (1k steps) --lr=10/1 (alcanza 0.86 en la validación. - loss:1.23e-4)
*   TrackNet-PadelCleanAll2: Continuación, 25epochs (1k steps) --lr=1 (alcanza 0.86 en la validación. - loss:1.08e-4)
*   TrackNet-PadelCleanAll3: Continuación, 50epochs (1k steps) --lr=1 (alcanza 0.88 en la validación. - loss:1.06e-4)




#### Train desordenado
*    TrackNet-SoloPadelCleanAll: solo padel CleanDS, 50epochs (800) (valTrain=0.9, loss=2.3e-4, valTest=0.8)
-------
*    TrackNet-SoloPadelCleanAll-720p: solo padel CleanDS (480p), 35epochs (800) (valTrain=0.9, loss=1.9e-4)
*    TrackNet-SoloPadelCleanAll-720p2: Continuación, +50epochs (1500) (valTrain=0.9, loss=1.75e-4)
-------
*    TrackNet-SoloPadelCleanAll-480p: solo padel CleanDS (480p) sigma=3, 30epoch (1500) (valTrain=0.9, loss=1.29e-4, cleanVal=0.9-0.94)
-------
*    TrackNet-SoloPadelCleanAll-480p-5F+2F: solo padel CleanDS (480p) sigma=3, 22epoch(1500) (valTrain=0.92, loss=1.25e-4, )

In [ ]:
weights_path = '/content/gdrive/MyDrive/PadelModels/TrackNetWeights2/TrackNet-SoloPadelCleanAll-480p-5F+2F'
!python train2.py --save_weights={weights_path}  --batch_size=2 --frame_stack=5 --back_frame_stack=2 --HEIGHT=480 --WIDTH=720 --lr=10 --tol=5  --pre_trained=True  --load_weights={weights_path}

lerning rate: 10.0
2022-06-09 15:36:34.725265: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Beginning training......
==========Epoch 0, Train steps: 1500, Learning rate: 10.0000==========
1500/1500 [==============================] - 596s 384ms/step - loss: 1.4676e-04 - binary_accuracy: 0.9998
==========Epoch 0 start validation========== 400
Epoch 0 accuracy: 0.856
Epoch 0 precision: 0.918
Epoch 0 recall: 0.909
Epoch 0 average = (accuracy + precision + recall)/3: 0.894
==========Epoch 1, Train steps: 1500, Learning rate: 10.0000==========
1500/1500 [==============================] - 577s 384ms/step - loss: 1.4688e-04 - binary_accuracy: 0.9998
==========Epoch 2, Train steps: 1500, Learning rate: 10.0000==========
1500/1500 [==============================] - 576s 384ms/step - loss: 1.3819e-04 - binary_accuracy: 0.9998
==========Epoch 3, Train ste

### Debug

In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [81]:
from utils import *
HEIGHT = 288
WIDTH = 512
BATCH_SIZE = 2
FRAME_STACK = 3
BACK_FRAME_STACK = 1

match_path = 'train_data'
match_list = [os.sep.join([os.getcwd(), match_path, match]) for match in os.listdir(match_path)]

x_train, x_test, y_train, y_test = split_train_test2(match_list, FRAME_STACK, BACK_FRAME_STACK, ratio=0.9, shuffle=True)
gen = data_generator2(BATCH_SIZE, x_train+x_test, y_train+y_test, FRAME_STACK, BACK_FRAME_STACK)

18572
2064
18572


In [64]:
x, y = next(gen)
print(x.shape)

(2, 3, 288, 512)


In [65]:
list(zip(x_train, y_train))[:3]

[('/content/TrackNet-Padel/train_data/match22/x_data/match22_PremierRoma_Semis_LGBC_1_00_6_105.jpg',
  '/content/TrackNet-Padel/train_data/match22/y_data/match22_PremierRoma_Semis_LGBC_1_00_6_105.jpg'),
 ('/content/TrackNet-Padel/train_data/match20/x_data/match20_PremierRoma_Semis_LGBC_1_00_2_606.jpg',
  '/content/TrackNet-Padel/train_data/match20/y_data/match20_PremierRoma_Semis_LGBC_1_00_2_606.jpg'),
 ('/content/TrackNet-Padel/train_data/match53/x_data/match53_PremierRoma_Semis_LGBC_2_35_2_117.jpg',
  '/content/TrackNet-Padel/train_data/match53/y_data/match53_PremierRoma_Semis_LGBC_2_35_2_117.jpg')]

In [89]:
im = cv2.imread('train_data/match1/x_data/match1_PremierRoma_Octavos_GRMS_2_53_0_0.jpg')
im.shape

(288, 512, 3)